# Genetic Algorithm Approach

In this task, we will implement a **genetic algorithm (GA)**—a nature-inspired optimization technique modeled on the principles of natural selection and genetics. This algorithm iteratively evolves a population of candidate solutions toward an optimal or near-optimal solution for a given problem.

The process involves generating **offspring through recombination (crossover)** of the fittest individuals from the current generation. These offspring then undergo **mutation**, introducing genetic diversity and helping the algorithm avoid local optima.

This evolutionary cycle of **selection**, **recombination**, and **mutation** continues over multiple generations until convergence criteria are met—such as reaching a satisfactory fitness level or a maximum number of generations.

We begin by importing the necessary libraries.

In [57]:
import random
import numpy as np

Before executing the genetic algorithm, we need to define several key parameters that govern the behavior and performance of the evolutionary process. These parameters control how the population evolves over time and directly affect the convergence speed and quality of the final solution.

In [127]:
mu = 5 # number of parents
lambd = 600  # number of offspring
num_generations = 2000 # number of generations

The next step is to define the matrix that we are going to study, we are going to prove the algorithm with two case: one with 26 cities an other with 42.

In [ ]:
with open('data/fri26_d.txt') as f:
    tsp_26 = [[int(num) for num in line.split()] for line in f]
# Best solution: 937

with open('data/dantzig42_d.txt') as f:
    tsp_42 = [[int(num) for num in line.split()] for line in f]
# Best solution: 10628

## Generating an Initial Random Solution

To begin solving the Traveling Salesman Problem (TSP) using a genetic algorithm, we first need a way to generate an initial population of candidate solutions. Each solution represents a unique route that visits every city exactly once.

The `randomSolution` function provides this capability by producing a valid, randomly ordered list of cities—essentially a permutation of all the cities in the problem instance.

In [ ]:
def randomSolution(tsp):
    """
    Generate a random solution for the Traveling Salesman Problem (TSP).
    
    This function creates a random permutation of the cities, representing
    a potential route that visits each city exactly once.
    
    Parameters:
    tsp (list): The distance matrix representing the TSP instance.
    
    Returns:
    list: A randomly ordered list of city indices representing a route.
    """
    solution = random.sample(range(len(tsp)), len(tsp))
    return solution

## Evaluating a Solution: Route Length Calculation

Once we generate a potential solution to the Traveling Salesman Problem (TSP), we need a way to evaluate its quality. The `routeLength` function calculates the **total distance** of a given path (solution), which serves as the **fitness** of that solution in the context of the genetic algorithm.

In [ ]:
def routeLength(tsp, solution):
    """
    Calculate the total distance of a given route (solution) for the Traveling Salesman Problem (TSP).

    Parameters:
    tsp (list): The distance matrix representing the TSP instance.
    solution (list): A list of city indices representing a route.

    Returns:
    int: The total distance of the route.
    """
    routeLength = 0
    for i in range(len(solution)):
        routeLength += tsp[solution[i-1]][solution[i]]
    return routeLength

## Mutation Operator: 2-Opt Swap

Mutation is a key operator in genetic algorithms that helps maintain genetic diversity within the population, allowing the algorithm to explore new parts of the solution space and avoid premature convergence.

In the context of the Traveling Salesman Problem (TSP), one effective mutation technique is the **2-opt swap**. This operation selects two indices in the route and reverses the order of the cities between them, producing a new, slightly modified solution.

In [ ]:
def mutation(route):
    """
    Perform a 2-opt mutation on a given route.

    This function selects two random indices in the route and reverses the order
    of the cities between them, creating a new, slightly modified route.

    Parameters:
    route (list): A list of city indices representing a route.

    Returns:
    list: A new route with the 2-opt mutation applied.
    """
    route_list = list(route)
    i, k = np.random.randint(0, len(route), size = 2)
    if i > k:
        i, k = k, i
        
    new_route = route_list[:i] + route_list[i:k+1][::-1] + route_list[k+1:]
    return new_route

## Crossover Operator: Order Crossover (OX)

In genetic algorithms, the **crossover (recombination)** operation is a fundamental step used to combine the genetic information of two parent solutions to generate new offspring. This operator enables the algorithm to explore the solution space by mixing features of high-performing individuals.

For the **Traveling Salesman Problem (TSP)**, where each solution is a permutation of cities, standard crossover methods are not suitable because they can produce invalid offspring (e.g., with repeated cities). To solve this, we use a permutation-preserving crossover method.

### Order Crossover (OX) — Proposed by Davis

The **Order Crossover (OX)** operator preserves the relative order and position of cities from both parents. It works as follows:

1. Select a subsequence (subtour) randomly from one parent.
2. Copy it into the corresponding position in the offspring.
3. Fill the remaining positions with the genes (cities) from the other parent in the order they appear, skipping those already present.


In [ ]:
def recombine(parent1, parent2):
    """
    Perform Order Crossover (OX) on two parent solutions.

    This function implements the Order Crossover (OX) operator, which is used in genetic algorithms
    to combine two parent solutions into two offspring while preserving the relative order of genes.

    Parameters:
    parent1 (list): The first parent solution, represented as a list of city indices.
    parent2 (list): The second parent solution, represented as a list of city indices.

    Returns:
    tuple: A tuple containing two offspring solutions (child1, child2), each represented as a list of city indices.
    """
    length = len(parent1)  # Obtener la longitud de los padres
    
    # Seleccionar dos puntos de corte aleatorios
    point1 = random.randint(0, length - 1)
    point2 = random.randint(0, length - 1)
    
    # Asegurarse de que point1 sea menor que point2
    if point1 > point2:
        point1, point2 = point2, point1
    
    # Crear los hijos
    child1 = [-1] * length
    child2 = [-1] * length
    
    # Copiar el segmento seleccionado del padre 1 a los hijos
    child1[point1:point2+1] = parent1[point1:point2+1]
    child2[point1:point2+1] = parent2[point1:point2+1]
    
    # Obtener los genes faltantes en los hijos preservando el orden relativo
    genes_left1 = [gene for gene in parent2 if gene not in child1]
    genes_left2 = [gene for gene in parent1 if gene not in child2]
    
    # Rellenar los genes faltantes en los hijos con los genes restantes
    index1 = (point2 + 1) % length
    index2 = (point2 + 1) % length
    
    for _ in range(length - (point2 - point1 + 1)):
        if child1[index1] == -1:
            child1[index1] = genes_left1.pop(0)
        if child2[index2] == -1:
            child2[index2] = genes_left2.pop(0)
        index1 = (index1 + 1) % length
        index2 = (index2 + 1) % length
    
    return child1, child2

## Main Evolutionary Process

The `calculate_optim` function is the core of the genetic algorithm implementation for solving the **Traveling Salesman Problem (TSP)**. It orchestrates the evolutionary process over multiple generations, applying **selection**, **crossover**, and **mutation** to evolve a population of candidate solutions toward an optimal tour.

This function requires:
- A **distance matrix** (`tsp`) representing the pairwise distances between cities.
- Predefined global parameters:
  - `mu`: number of parents selected.
  - `lambd`: number of offspring generated.
  - `num_generations`: total number of generations to evolve.

In [ ]:
def calculate_optim(tsp):
    """
        Perform optimization using a genetic algorithm to solve the Traveling Salesman Problem (TSP).

        This function evolves a population of candidate solutions over multiple generations
        using selection, crossover, and mutation to find an optimal or near-optimal solution.

        Parameters:
        tsp (list): The distance matrix representing the TSP instance.

        Returns:
        tuple: A tuple containing the best solution (list of city indices) and its total distance (int).
    """
    Best = []
    P = np.array([randomSolution(tsp) for i in range(lambd)])

    for generation in range(num_generations):
        child_distances = []

        for children in P:
            children_distance = routeLength(tsp, children)
            child_distances.append(children_distance)

            if len(Best)==0 or children_distance < routeLength(tsp, Best) :
                Best = children

        # Order the distances and extract the indexes   
        tuple_distances = [(value, index) for index, value in enumerate(child_distances)]
        tuple_distances_ordered = sorted(tuple_distances, key=lambda x:x[0])
        indexes = [index for value, index in tuple_distances_ordered]

        # Select the best mu individuals
        Q = P[indexes[:2*mu]]

        # Create the new generation
        P = np.empty((2*lambd, len(tsp)), dtype=int)
        aux = 0
        for i in range(mu):
            for j in range(lambd // mu):
                parent1 = np.copy(Q[i])
                parent2 = np.copy(Q[i+mu])
                child1, child2 = recombine(parent1, parent2)
                child1 = mutation(child1)
                P[aux] = child1
                aux += 1
                child2 = mutation(child2)
                P[aux] = child2
                aux += 1    
    
    best_length = routeLength(tsp, Best)
    
    return Best, best_length

First, we prove the algorithm with the 26 cities problem.

In [128]:
Best, best_length = calculate_optim(tsp_26)

print("Best: ", Best)
print("Best Final distance: ", best_length)

# Best solution found: 937
# Best solution: 937

Best:  [12 11 10 15 18 19 17 16 20 21 25 22 23 24  0  1  2  3  5  4  6  7  8  9
 13 14]
Best Final distance:  937


Now, with the 42 cities problem.

In [129]:
Best, best_length = calculate_optim(tsp_42)

print("Best: ", Best)
print("Best Final distance: ", best_length)

# Best solution found: 699
# Best solution: 699

Best:  [ 0 41 40 39 38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19
 18 17 16 15 14 13 12 11 10  9  8  7  6  5  4  3  2  1]
Best Final distance:  699


## Summary

The results demonstrate that the genetic algorithm effectively finds optimal solutions across both test cases, regardless of the problem size. 

- **Case 1 (26 cities):**  
  The algorithm successfully identified the optimal route in a relatively short time. Interestingly, the `(μ, λ)` strategy without recombination converged faster in this smaller instance, indicating that recombination might not always be necessary for simpler problems.

- **Case 2 (42 cities):**  
  The algorithm also reached the optimal solution in this larger instance, showcasing the strength of the recombination operator. The inclusion of crossover (specifically, the Order Crossover method) played a crucial role in enhancing the search capability for this more complex scenario.

These observations underline the importance of **parameter tuning**. Adjusting the values of `λ` (offspring size), `μ` (number of parents), and the number of generations is essential to balance **exploration** and **exploitation** within the algorithm. Proper tuning can significantly impact convergence speed and solution quality.

---

### Importance of Recombination Before Mutation

Applying recombination before mutation in a genetic algorithm brings several key advantages:

1. **Enhanced Exploration of the Search Space:**  
   Recombination combines valuable traits from different individuals, generating offspring that may inherit the strengths of both parents. This promotes diversity in the population and helps avoid premature convergence to local optima.

2. **Preservation of Useful Genetic Information:**  
   Without recombination, mutation alone might randomly disrupt beneficial traits. Recombination allows these features to be retained and reused in the evolutionary process, ensuring that high-quality genetic material is not lost.

3. **Faster Convergence to Optimal Solutions:**  
   By producing new individuals with promising genetic combinations, recombination can speed up the search process. It increases the likelihood of generating high-quality solutions early, reducing the number of generations needed to converge.

---

### Conclusion

The **order of genetic operators** is crucial for the performance of evolutionary algorithms. Applying **recombination before mutation** enables:
- Better search space coverage
- Preservation of advantageous structures
- Faster and more reliable convergence

These effects contribute to a more robust and efficient genetic algorithm capable of solving complex combinatorial problems like the Traveling Salesman Problem.
